In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored , concordance_index_ipcw
from sklearn.impute import SimpleImputer
from sksurv.util import Surv


In [2]:
# Clinical Data
df = pd.read_csv("./X_train/clinical_train.csv")
df_eval = pd.read_csv("./X_test/clinical_test.csv")

# Molecular Data
maf_df = pd.read_csv("./X_train/molecular_train.csv")
maf_eval = pd.read_csv("./X_test/molecular_test.csv")

target_df = pd.read_csv("./target_train.csv")

df.head()
maf_df.head()

,ID,CHR,START,END,REF,ALT,GENE,PROTEIN_CHANGE,EFFECT,VAF,DEPTH
0,P100000,11,119149248.0,119149248.0,G,A,CBL,p.C419Y,non_synonymous_codon,0.0830,1308.0
1,P100000,5,131822301.0,131822301.0,G,T,IRF1,p.Y164*,stop_gained,0.0220,532.0
2,P100000,3,77694060.0,77694060.0,G,C,ROBO2,p.?,splice_site_variant,0.4100,876.0
3,P100000,4,106164917.0,106164917.0,G,T,TET2,p.R1262L,non_synonymous_codon,0.4300,826.0
4,P100000,2,25468147.0,25468163.0,ACGAAGAGGGGGTGTTC,A,DNMT3A,p.E505fs*141,frameshift_variant,0.0898,942.0


In [3]:
import re
def parse_protein_change(protein_change):
    text = str(protein_change)

    # Cas 1 : classique ou stop codon (ex: p.R882C, p.Y164*, p.E774X)
    match1 = re.match(r"p\.([A-Z])(\d+)([A-Z*X])", text)
    if match1:
        return pd.Series(match1.groups())

    # Cas 2 : frameshift avec stop (ex: p.E505fs*141)
    match2 = re.match(r"p\.([A-Z])(\d+)fs\*?\d*", text)
    if match2:
        aa_from, pos = match2.groups()
        return pd.Series([aa_from, pos, 'fs'])

    # Cas 3 : frameshift simple (ex: p.E856fs)
    match3 = re.match(r"p\.([A-Z])(\d+)fs", text)
    if match3:
        aa_from, pos = match3.groups()
        return pd.Series([aa_from, pos, 'fs'])

    # Cas 4 : mutation intronique ou notation c. (ignorer)
    return pd.Series([None, None, None])

# Appliquer à ta colonne
maf_df[['aa_from', 'mutation_position', 'aa_to']] = maf_df['PROTEIN_CHANGE'].apply(parse_protein_change)
maf_eval[['aa_from', 'mutation_position', 'aa_to']] = maf_eval['PROTEIN_CHANGE'].apply(parse_protein_change)

In [4]:
maf_eval[["PROTEIN_CHANGE","aa_from"]]

,PROTEIN_CHANGE,aa_from
0,p.K57E,K
1,p.K57E,K
2,p.K57E,K
3,p.K57E,K
4,p.Q754X,Q
...,...,...
3084,MLL_PTD,None
3085,MLL_PTD,None
3086,MLL_PTD,None
3087,MLL_PTD,None


In [5]:

target_df.dropna(subset=['OS_YEARS', 'OS_STATUS'], inplace=True)


target_df['OS_YEARS'] = pd.to_numeric(target_df['OS_YEARS'], errors='coerce')
target_df['OS_STATUS'] = target_df['OS_STATUS'].astype(bool)

print(target_df[['OS_STATUS', 'OS_YEARS']].dtypes)

features = ['BM_BLAST', 'HB', 'PLT','ANC','MONOCYTES']
target = ['OS_YEARS', 'OS_STATUS']

X = df.loc[df['ID'].isin(target_df['ID']), features]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', target_df)

OS_STATUS       bool
OS_YEARS     float64
dtype: object


In [6]:
from functions import has_pattern, count_anomalies

df['anc_wbc_ratio'] = df['ANC'] / (df['WBC'] + 1e-3)
df_eval['anc_wbc_ratio'] = df_eval['ANC'] / (df_eval['WBC'] + 1e-3)
df['blast_hb_ratio'] = df['BM_BLAST'] / (df['HB'] + 1e-3) 
df_eval['blast_hb_ratio'] = df_eval['BM_BLAST'] / (df_eval['HB'] + 1e-3) 


df['female'] = df['CYTOGENETICS'].apply(lambda x: has_pattern(x, r'46,?xx(\[\d+\])?'))
df['male'] = df['CYTOGENETICS'].apply(lambda x: has_pattern(x, r'46,?xy(\[\d+\])?'))
df_eval['female'] = df_eval['CYTOGENETICS'].apply(lambda x: has_pattern(x, r'46,?xx(\[\d+\])?'))
df_eval['male'] = df_eval['CYTOGENETICS'].apply(lambda x: has_pattern(x, r'46,?xy(\[\d+\])?'))

df['has_chr7_deletion'] = df['CYTOGENETICS'].str.contains(r'del\(7\)|-7', na=False).astype(int)
df_eval['has_chr7_deletion'] = df_eval['CYTOGENETICS'].str.contains(r'del\(7\)|-7', na=False).astype(int)
df['has_chr5_deletion'] = df['CYTOGENETICS'].str.contains(r'del\(5\)|-5', na=False).astype(int)
df_eval['has_chr5_deletion'] = df_eval['CYTOGENETICS'].str.contains(r'del\(5\)|-5', na=False).astype(int)

df['translocation_count'] = df['CYTOGENETICS'].apply(lambda x: count_anomalies(x, 't'))
df['deletion_count'] = df['CYTOGENETICS'].apply(lambda x: count_anomalies(x, 'del'))
df['derivative_count'] = df['CYTOGENETICS'].apply(lambda x: count_anomalies(x, 'der'))
df['addition_count'] = df['CYTOGENETICS'].apply(lambda x: count_anomalies(x, 'add'))

df_eval['translocation_count'] = df_eval['CYTOGENETICS'].apply(lambda x: count_anomalies(x, 't'))
df_eval['deletion_count'] = df_eval['CYTOGENETICS'].apply(lambda x: count_anomalies(x, 'del'))
df_eval['derivative_count'] = df_eval['CYTOGENETICS'].apply(lambda x: count_anomalies(x, 'der'))
df_eval['addition_count'] = df_eval['CYTOGENETICS'].apply(lambda x: count_anomalies(x, 'add'))

df['cytogenetic_length'] = df['CYTOGENETICS'].fillna('').str.len()
df_eval['cytogenetic_length'] = df_eval['CYTOGENETICS'].fillna('').str.len()




### Testing the RSF predictor:

In [7]:
from functions import is_pattern, has_complex
from sklearn.impute import SimpleImputer

eval_genes = set(maf_eval['GENE'])
eval_effects = set(maf_eval['EFFECT'])
eval_chr = set(maf_eval['CHR'])
eval_aa_from = set(maf_eval['aa_from'].fillna('complex_aa'))

gene_features = pd.get_dummies(maf_df['GENE'], prefix='gene')
effect_features = pd.get_dummies(maf_df['EFFECT'], prefix='effect')
chr_features = pd.get_dummies(maf_df['CHR'], prefix='CHR')
#aa_from_features = pd.get_dummies(maf_df['aa_from'].fillna('complex_aa'), prefix='aa')

filtered_gene_features = gene_features[[col for col in gene_features.columns if col[5:] in eval_genes]]
filtered_effect_features = effect_features[[col for col in effect_features.columns if col[7:] in eval_effects]]
filtered_chr_features = chr_features[[col for col in chr_features.columns if col[4:] in eval_chr]]
#filtered_aa_features = aa_from_features[[col for col in aa_from_features.columns if col[3:] in eval_aa_from]]

molecular_features = pd.concat([filtered_gene_features, filtered_effect_features, filtered_chr_features ], axis=1)
molecular_features_grouped = molecular_features.groupby(maf_df['ID']).max()


maf_df['length'] = maf_df['END'] - maf_df['START']
vaf = maf_df[['VAF', 'DEPTH', 'length']].groupby(maf_df['ID']).max()

prot = maf_df[['PROTEIN_CHANGE']].copy()
prot['is_frameshift'] = prot['PROTEIN_CHANGE'].apply(lambda x : is_pattern(x, 'fs')).astype(int)
prot['is_nonsense'] = prot['PROTEIN_CHANGE'].apply(lambda x : is_pattern(x, '*')).astype(int)

driver_genes = ["TP53", "NPM1", "FLT3", "DNMT3A", "IDH1", "IDH2", "RUNX1", "TET2"]
driver = maf_df["GENE"].isin(driver_genes).groupby(maf_df['ID']).sum().rename('driver')

driver_effects = ["frameshift_variant", "nonsense_variant", "splice_site_variant"]
driver_effects = maf_df["EFFECT"].isin(driver_effects).groupby(maf_df['ID']).sum().rename('driver_effect')


prot = prot[['is_frameshift', 'is_nonsense']].groupby(maf_df['ID']).max()

tmp = maf_df.groupby('ID').size().reset_index(name='Nmut')
high_vaf_count = maf_df[maf_df["VAF"] > 0.6].groupby("ID").size().rename("Nmut_high_VAF")

df = df.merge(tmp, on='ID', how='left').fillna({'Nmut': 0})
df = df.merge(high_vaf_count, on = 'ID', how = 'left').fillna(0)
df = df.merge(driver, on = 'ID', how = 'left').fillna(0)
#df = df.merge(driver_effects, on = 'ID', how = 'left').fillna(0)

merged_df = pd.merge(df, molecular_features_grouped, on='ID', how='left')
merged_df = pd.merge(merged_df, vaf, on='ID', how='left')
merged_df = pd.merge(merged_df, prot, on='ID', how='left')


merged_df = merged_df.fillna(0)

clinical_features = ['BM_BLAST', 'HB', 'PLT', 'female', 'has_chr7_deletion', 'has_chr5_deletion', 'translocation_count', 
                     'deletion_count', 'derivative_count', 'addition_count', 'anc_wbc_ratio', 'blast_hb_ratio']

mol_features = ['VAF', 'DEPTH', 'is_frameshift', 'is_nonsense', 'length', 'Nmut', 'Nmut_high_VAF', 'driver']
all_features = clinical_features + mol_features + list(molecular_features_grouped.columns)


X = merged_df.loc[merged_df['ID'].isin(target_df['ID']), all_features]
y = target_df.loc[target_df['ID'].isin(merged_df['ID']), ['OS_STATUS', 'OS_YEARS']]

X = X.apply(pd.to_numeric, errors='coerce')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
pd.options.display.max_columns = 200
df

,ID,CENTER,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS,anc_wbc_ratio,female,male,has_chr7_deletion,has_chr5_deletion,translocation_count,deletion_count,derivative_count,addition_count,cytogenetic_length,Nmut,Nmut_high_VAF,driver
0,P132697,MSK,14.0,2.80,0.20,0.70,7.6,119.0,"46,xy,del(20)(q12)[2]/46,xy[18]",0.071403,0.0,1.0,0,0,0,1,0,0,31,9.0,0.0,5.0
1,P132698,MSK,1.0,7.40,2.40,0.10,11.6,42.0,"46,xx",0.324281,1.0,0.0,0,0,0,0,0,0,5,3.0,0.0,1.0
2,P116889,MSK,15.0,3.70,2.10,0.10,14.2,81.0,"46,xy,t(3;3)(q25;q27)[8]/46,xy[12]",0.567414,0.0,1.0,0,0,1,0,0,0,34,3.0,0.0,0.0
3,P132699,MSK,1.0,3.90,1.90,0.10,8.9,77.0,"46,xy,del(3)(q26q27)[15]/46,xy[5]",0.487055,0.0,1.0,0,0,0,1,0,0,33,11.0,0.0,1.0
4,P132700,MSK,6.0,128.00,9.70,0.90,11.1,195.0,"46,xx,t(3;9)(p13;q22)[10]/46,xx[10]",0.075781,1.0,0.0,0,0,1,0,0,0,35,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3318,P121828,VU,1.0,3.70,2.53,0.53,8.9,499.0,"46,xy[20]",0.683599,0.0,1.0,0,0,0,0,0,0,9,2.0,0.0,0.0
3319,P121829,VU,0.0,4.20,2.40,0.22,10.6,49.0,"46,xy,del(13)(q12q14)[1]/45,x,-y,del(13)(q12q1...",0.571293,0.0,1.0,0,0,0,2,0,0,61,1.0,0.0,0.0
3320,P121830,VU,0.0,1.80,0.55,0.29,9.4,86.0,"46,xy,del(20)(q11.2q13.1)[4]/45,xy,idem,-7[16]",0.305386,0.0,1.0,1,0,0,1,0,0,46,6.0,0.0,1.0
3321,P121853,VU,5.0,1.37,0.37,0.11,11.4,102.0,"46,xx,del(1)(p34)[5]/45,xx,sl,-18[12]/46,xx,sd...",0.269876,1.0,0.0,0,0,0,1,0,0,62,4.0,0.0,3.0


In [8]:
from sksurv.ensemble import RandomSurvivalForest

y_train_surv = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', y_train)
y_test_surv = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', y_test)


rsf = RandomSurvivalForest(
    n_estimators=1000, 
    min_samples_split=100,  
    min_samples_leaf=50, 
    max_features="sqrt", 
    n_jobs=-1,  
    random_state=42  
)

# Fit the model on training data
rsf.fit(X_train, y_train_surv)

RandomSurvivalForest(min_samples_leaf=50, min_samples_split=100,
                     n_estimators=1000, n_jobs=-1, random_state=42)

In [9]:
cindex_test = concordance_index_ipcw(y_test_surv, y_test_surv, rsf.predict(X_test), tau=None)[0]
print(f"C-index (test): {cindex_test:.3f}")

C-index (test): 0.682


In [11]:
print(X_train.columns)

Index(['BM_BLAST', 'HB', 'PLT', 'female', 'has_chr7_deletion',
       'has_chr5_deletion', 'translocation_count', 'deletion_count',
       'derivative_count', 'addition_count',
       ...
       'CHR_21', 'CHR_22', 'CHR_3', 'CHR_4', 'CHR_5', 'CHR_6', 'CHR_7',
       'CHR_8', 'CHR_9', 'CHR_X'],
      dtype='object', length=113)


In [10]:
gene_features_eval = pd.get_dummies(maf_eval['GENE'], prefix='gene')
effect_features_eval = pd.get_dummies(maf_eval['EFFECT'], prefix='effect')
chr_features_eval = pd.get_dummies(maf_eval['CHR'], prefix='CHR')
#aa_from_features_eval = pd.get_dummies(maf_eval['aa_from'].fillna('complex_aa'), prefix='aa')

molecular_features_eval = pd.concat([gene_features_eval, effect_features_eval, chr_features_eval], axis=1)


molecular_features_grouped_eval = molecular_features_eval.groupby(maf_eval['ID']).max()

maf_eval['length'] = maf_eval['END'] - maf_eval['START']
vaf_eval = maf_eval[['VAF', 'DEPTH', 'length']].groupby(maf_eval['ID']).max()


driver_eval = maf_eval["GENE"].isin(driver_genes).groupby(maf_eval['ID']).sum().rename('driver')
driver_effects_eval = maf_eval["EFFECT"].isin(driver_effects).groupby(maf_eval['ID']).sum().rename('driver_effect')

prot_eval = maf_eval[['PROTEIN_CHANGE']].copy()
prot_eval['is_frameshift'] = prot_eval['PROTEIN_CHANGE'].apply(lambda x : is_pattern(x, 'fs'))
prot_eval['is_nonsense'] = prot_eval['PROTEIN_CHANGE'].apply(lambda x : is_pattern(x, '*'))


prot_eval = prot_eval[['is_frameshift', 'is_nonsense']].groupby(maf_eval['ID']).max()

tmp_eval = maf_eval.groupby('ID').size().reset_index(name='Nmut')
high_vaf_count_eval = maf_eval[maf_eval["VAF"] > 0.6].groupby("ID").size().rename("Nmut_high_VAF")

df_eval = df_eval.merge(tmp_eval, on='ID', how='left').fillna({'Nmut': 0})
df_eval = df_eval.merge(high_vaf_count_eval, on = 'ID', how = 'left').fillna({'Nmut': 0})
df_eval = df_eval.merge(driver_eval, on = 'ID', how = 'left').fillna(0)
#df_eval = df_eval.merge(driver_effects_eval, on = 'ID', how = 'left').fillna(0)

merged_df_eval = pd.merge(df_eval, molecular_features_grouped_eval, on='ID', how='left')
merged_df_eval = pd.merge(merged_df_eval, vaf_eval, on='ID', how='left')
merged_df_eval = pd.merge(merged_df_eval, prot_eval, on='ID', how='left')



merged_df_eval = merged_df_eval.fillna(0)


# Select features
clinical_features_eval = ['BM_BLAST', 'HB', 'PLT', 'female','has_chr7_deletion', 'has_chr5_deletion','translocation_count', 
                     'deletion_count', 'derivative_count', 'addition_count', 'anc_wbc_ratio', 'blast_hb_ratio']

mol_features_eval = ['VAF', 'DEPTH','is_frameshift', 'is_nonsense','length', 'Nmut', 'Nmut_high_VAF', 'driver']
all_features_eval = clinical_features_eval + mol_features_eval+ list(molecular_features_grouped_eval.columns)


X_eval = merged_df_eval.loc[:, all_features_eval]
X_eval = X_eval.apply(pd.to_numeric, errors='coerce')


X_eval = X_eval[all_features]


rsf = RandomSurvivalForest(
    n_estimators=1000,
    min_samples_split=100,
    min_samples_leaf=50,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

# Fit the model on training data
rsf.fit(X, Surv.from_dataframe('OS_STATUS', 'OS_YEARS', y))

# Predict risk scores for the evaluation data
risk_scores_eval = rsf.predict(X_eval)


In [12]:
pd.options.display.max_columns = 200
(X_eval['Nmut_high_VAF'] > 0).mean()

np.float64(0.20368818105616093)

In [11]:
print(risk_scores_eval)

[857.1310028  999.75115325 431.79644229 ... 707.61899463 464.76915761
 817.17893174]


### FIRST SUBMISSION


In [12]:
import pandas as pd


ids = df_eval['ID']
risk_scores = rsf.predict(X_eval)

# Create a DataFrame with IDs and risk scores
df_submission = pd.DataFrame({'ID': ids, 'risk_score': risk_scores})

# Save the DataFrame to a CSV file
df_submission.to_csv('sub_BM30.csv', index=False)
